In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import io
from pprint import pprint
from time import time
from pprint import pprint

from google.cloud import vision
from google.cloud.vision import types
import matplotlib.pyplot as plt

from context import backend
from backend.apps.detections import fix_ros_path  # needs to be before cv2 import
fix_ros_path.fix_ros_cv_path()
from backend.apps.detections.gcv_api import VisionClient 
from backend.apps.detections.cv_helpers import image_resize
from backend.apps.detections.gcv_api import bb_norm_to_corner_pixels, objects_dict_coords_to_pic
from backend.apps.detections import object_detection
import cv2
import torch
from PIL import Image

In [ ]:
DIR = '/home/matt/polybox/Fotos'
FILE_NAME = 'lessdark.jpg'
FILE_PATH = os.path.join(DIR, FILE_NAME)

def read_img_bin(path):
    with io.open(path, 'rb') as image_file:
        return image_file.read()
    
def read_img_cv(path):
    return cv2.imread(path)

In [ ]:
def demo_gcp_demo():
    DIR = '/home/matt/polybox/Fotos/Yb/'
    FILES = ['less.jpg']

    files = FILES if len(FILES) != 0 else os.listdir(DIR)
    paths = [os.path.join(DIR, f) for f in files]

    for path in paths:
        print(path)
        img_cv = read_img_cv(path)
        img_bin = read_img_bin(path)
        vision_client = VisionClient()
        objects = vision_client.localize_objects(img_bin)
        # objects_dict_coords_to_pic(objects, img_cv.shape)
        pprint(objects)
        show_detections(img_cv, objects)
        cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
torch.cuda.empty_cache()
img_path = '/home/matt/polybox/Fotos/Yb/20191116_010804.jpg'
img_pil = Image.open(img_path)
objects = object_detection.detect_objects(img_pil)
boxes, pred_cls, pred_score = object_detection.get_prediction(img_pil, 0.5)
pprint(objects)

In [ ]:
fig, ax = plt.subplots()
for det in objects:
    vertices = det['bb_vertices']
    
    pprint(vertices)
    #vertices +=  det['bb_vertices'][0]
    x = [vertex['x'] for vertex in vertices]
    y = [vertex['y'] for vertex in vertices]
    ax.plot(x, y)
    

In [ ]:
object_detection.plot_results(img_pil, threshold=0.2)